In [1]:
!pip install pandas
!pip install openpyxl

Access is denied.
Access is denied.


In [2]:
import pandas as pd

In [3]:
general_information = pd.read_excel('data/general_information_bkp.xlsx')
type_and_level = pd.read_excel('data/type_and_level_bk.xlsx')
geographical_facilities = pd.read_excel('data/geographical_facilities_bkp.xlsx')
buildings_and_library = pd.read_excel('data/buildings_and_library.xlsx')
buildings = pd.read_excel('data/buildings_bkp.xlsx')
compliance1 = pd.read_excel('data/compliance_1_bkp.xlsx')
compliance2 = pd.read_excel('data/compliance_2_bkp.xlsx')
compliance3 = pd.read_excel('data/compliance_3_bkp.xlsx')
compliance4 = pd.read_excel('data/compliance_4_bkp.xlsx')


In [171]:
general_information = general_information[general_information['submitted']==1]

In [196]:
TYPE_SCHOOL = 1
TYPE_SCHOOL_AND_COLLEGE = 2
TYPE_COLELGE = 3
TYPE_MADRASHA = 4

In [216]:
from enum import Enum

class InstitutionType(Enum):
    SCHOOL = 1
    SCHOOL_AND_COLLEGE = 2
    MADRASHA = 3

In [217]:
from typing import Dict
institute_type_map_total_institution: Dict[int, int] = {}
institute_type_map_total_funded_institution: Dict[int, int] = {}
institute_type_map_total_refunded_institution: Dict[int, int] = {}

In [218]:
merged_eiin_institutes = pd.merge(general_information, type_and_level, on='eiin', how='inner')

In [246]:
df_total_schools = merged_eiin_institutes[merged_eiin_institutes['institution_type']==TYPE_SCHOOL]
df_total_school_and_college= merged_eiin_institutes[merged_eiin_institutes['institution_type'].isin([TYPE_SCHOOL_AND_COLLEGE, TYPE_COLELGE])]
df_total_madrasha = merged_eiin_institutes[merged_eiin_institutes['institution_type']== TYPE_MADRASHA]

In [247]:
institute_type_map_total_institution[InstitutionType.SCHOOL.value] = df_total_schools.shape[0]
institute_type_map_total_institution[InstitutionType.SCHOOL_AND_COLLEGE.value] = df_total_school_and_college.shape[0]
institute_type_map_total_institution[InstitutionType.MADRASHA.value] = df_total_madrasha.shape[0]

In [248]:
pbgsi_institutes = compliance2[(compliance2['pbgsi_scheme']==1)]

total_pgbsi_institutes = pbgsi_institutes.shape[0]

In [249]:
merged_general_information_pgbsi = pd.merge(general_information, pbgsi_institutes, on="eiin", how="inner")

In [250]:
df_type_and_level_secondary_school = type_and_level[type_and_level['institution_type']==TYPE_SCHOOL]

In [251]:
merged_general_information_pgbsi_secondary = pd.merge(merged_general_information_pgbsi, df_type_and_level_secondary_school, on = "eiin", how="inner")

In [252]:
total_secondary_school_pbgsi = merged_general_information_pgbsi_secondary.shape[0]
institute_type_map_total_funded_institution[InstitutionType.SCHOOL.value] = total_secondary_school_pbgsi

In [253]:
df_type_and_level_secondary_school_and_college = type_and_level[type_and_level["institution_type"].isin([TYPE_SCHOOL_AND_COLLEGE, TYPE_COLELGE]) ]

In [254]:
merged_general_information_pgbsi_school_and_college = pd.merge(merged_general_information_pgbsi, df_type_and_level_secondary_school_and_college, on='eiin', how='inner')

In [255]:
total_school_and_college = merged_general_information_pgbsi_school_and_college.shape[0]
institute_type_map_total_funded_institution[InstitutionType.SCHOOL_AND_COLLEGE.value] = total_school_and_college

In [256]:
df_type_and_level_madrasha = type_and_level[type_and_level['institution_type']==TYPE_MADRASHA]

In [257]:
merged_general_information_pgbsi_madrasha = pd.merge(merged_general_information_pgbsi, df_type_and_level_madrasha, on='eiin', how='inner')

In [258]:
total_madrasha = merged_general_information_pgbsi_madrasha.shape[0]
institute_type_map_total_funded_institution[InstitutionType.MADRASHA.value] = total_madrasha

In [259]:
import openpyxl
from openpyxl.utils import get_column_letter,get_column_letter
from openpyxl.styles import Alignment, Border, Side

In [260]:
print(institute_type_map_total_funded_institution)

{1: 5565, 2: 618, 3: 2021}


In [261]:
wb = openpyxl.Workbook()
ws = wb.active

ws['A1'] = 'Type of Institute'
ws.merge_cells('A1:A2')

ws ['B1'] = 'Received Grant'
ws.merge_cells('B1:C1')

ws['D1'] = 'Refunded Grant'
ws.merge_cells('D1:E1')

ws['B2'] = 'Number of Institute'
ws['C2'] = 'Percentage (%)'

ws['D2']= 'Number of Institute'
ws['E2'] = 'Percentage (%)'

row_num = 3

for inst_type in InstitutionType:
    institution_type_cell = "A"+str(row_num)
    ws[institution_type_cell] = inst_type.name

    print("institute type: "+inst_type.name)
    print("institution type value: "+str(inst_type.value))

    total_funded_institution_count = institute_type_map_total_funded_institution.get(inst_type.value, 0)
    print("total funded count: "+str(total_funded_institution_count))
    total_funded_instititon_count_cell = "B"+str(row_num)
    ws[total_funded_instititon_count_cell] = str(total_funded_institution_count)

    total_institution_count = institute_type_map_total_institution.get(inst_type.value)
    print("total institution count: "+str(total_institution_count))
    total_fund_percentage:float = 0.0 if total_funded_institution_count==0 else  total_funded_institution_count/total_institution_count
    total_fund_percentage_cell = "C"+str(row_num)
    ws[total_fund_percentage_cell] = str(total_fund_percentage)

    row_num+=1

center_alignment = Alignment(horizontal='center', vertical='center')
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for row in ws.iter_rows():
    for cell in row:
        cell.alignment = center_alignment
        cell.border = thin_border

for column_cells in ws.columns:
    length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in column_cells)
    column_letter = get_column_letter(column_cells[0].column)
    ws.column_dimensions[column_letter].width = length + 2  # Add some padding
    
wb.save('output/table_1_number_of_institutes_having_grand_and_refund.xlsx')

institute type: SCHOOL
institution type value: 1
total funded count: 5565
total institution count: 24733
institute type: SCHOOL_AND_COLLEGE
institution type value: 2
total funded count: 618
total institution count: 6086
institute type: MADRASHA
institution type value: 3
total funded count: 2021
total institution count: 11193
